In [7]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

### import data

def txt_to_df(filename):
    
    path = "/home/hermuba/res/data/data_from_dr_wu/"
    i = 0
    with open(path + filename) as f:
        for line in f:
            l = line.replace('\n', '').split('\t')
            if i == 0:
                l[0] = "Genome ID"
                l[1] = "Resistant Phenotype"
                df = pd.DataFrame(columns = l)
 
            else:
                ID = l[0]
                
                df.loc[i-1, :] = l
            i += 1
    return(df)

,Genome ID,Resistant Phenotype,Cluster 0,Cluster 1,Cluster 2,Cluster 3,Cluster 4,Cluster 5,Cluster 6,Cluster 7,...,Cluster 15940,Cluster 15941,Cluster 15942,Cluster 15943,Cluster 15944,Cluster 15945,Cluster 15946,Cluster 15947,Cluster 15948,Cluster 15949
0,1328432.3,Susceptible,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1328433.3,Resistant,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1328434.3,Resistant,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,1328442.3,Resistant,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1328448.3,Susceptible,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1328449.3,Resistant,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1328450.3,Susceptible,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1400156.3,Susceptible,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
8,1400157.3,Susceptible,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1400158.3,Susceptible,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
#those drugs have nearly 50 data, therefore their data are selected for training
train_drug = ['meropenem','cefepime','ceftazidime' 'gentamicin', 'ciprofloxacin',
              'trimethoprim_sulfamethoxaole', 'ampicillin', 'cefazolin', 'ampicillin_sulbactam']
prefix = ['card_pattern', 'gene_pattern', 'acc_pattern', 'acc_card_pattern']

In [23]:
# throw into naive bayes
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score     
def train_nb(df_ris):
    # hold out test set
    
    X_train, X_test, y_train, y_test = train_test_split(
        df_ris.drop(["Genome ID", "Resistant Phenotype"], axis = 1)
        , df_ris['Resistant Phenotype']
        , test_size=0.2 
        , random_state = 1
        , startify = df_ris['Resistant Phenotype']) 
    # test if random
    print(y_test)
    ### i dont really know pseudo random status mean (please read about it), random status 20 yields best result?? 0.7
    ### it saids it is "random sampling test set but it is not (WTF)
    
    X_train.reset_index(inplace = True, drop = True)
    y_train.reset_index(inplace = True, drop = True)
    
    # seperate train and test into k fold
    n=10
    cv = ShuffleSplit(n_splits=n, test_size=0.3, random_state = 0)
    ### n_split = 10, test_size, does not change result; random_state yeilds best result 0.75
    
    
    # bnb.fit
    bnb = BernoulliNB()
    val_score = 0
    for train_index, test_index in cv.split(X_train, y_train):      
        bnb.fit(X_train.iloc[train_index,:], y_train[train_index])
        val = bnb.predict(X_train.iloc[test_index])
        score = accuracy_score(y_train[test_index], val)
        val_score = val_score + score
    
    v = val_score/n
    
    # model.predict()
    
    test = accuracy_score(y_test, bnb.predict(X_test))
    
    return([v,test])
   

In [17]:
from sklearn import svm
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve
from sklearn.model_selection import StratifiedKFold # this is the problem!!

    
def train_SVM(df):
    
    # split test, train
    X_train, X_test, y_train, y_test = train_test_split(
        df.drop(["Genome ID", "Resistant Phenotype"], axis = 1)
        , df['Resistant Phenotype']
        , test_size=0.4 
        , random_state = 0)
    
    # choose estimator (our model)
    clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)
    
    # cross validation
    skf = StratifiedKFold(n_splits=3, random_state=None, shuffle=False)
    cv = skf.split(X_train, y_train)
   
    # tune hyperparameters
    gammas = np.logspace(-6, -1, 10)
    tuned_parameters = [{'kernel': ['rbf'], 'gamma': gammas,
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]
    classifier = GridSearchCV(estimator=clf, cv=cv, param_grid=tuned_parameters)
    classifier.fit(X_train.values, np.asarray(y_train))
    
    v = classifier.cv_results_['mean_test_score'][classifier.best_index_]
    param = classifier.best_params_
    print(param)
    test = classifier.score(X_test, y_test)
    
    return([v,test])

In [26]:
train_nb(txt_to_df("gene_pattern.trimethoprim_sulfamethoxazole"))

3     Resistant
2     Resistant
23    Resistant
39    Resistant
17    Resistant
29    Resistant
19    Resistant
35    Resistant
31    Resistant
Name: Resistant Phenotype, dtype: object


[0.60999999999999999, 0.77777777777777779]